# $\overline{\underline{\Large{\textrm{Clasificador de Persona Física o Moral}}}}$

## 0. Consideraciones

Este proyecto se centra en la implementación de un clasificador utilizando el modelo de Bayes Naive con el objetivo de distinguir entre personas físicas y morales a tráves de la cadena de caracteres que compone su nombre.

El conjunto de datos se ha construido utilizando información recopilada de "datamx.io" a través del siguiente enlace:

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ https://datamx.io/dataset/muestra-de-nombres-y-apellidos-comunes-en-mexico

Este conjunto de datos proporciona una muestra representativa de nombres y apellidos comunes en México, facilitando así el entrenamiento y evaluación del modelo de Bayes Naive.


Para los nombres de personas morales, se utilizaron datos abiertos del Sistema de Información Empresarial Mexicano, disponibles en:

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ https://datos.gob.mx/busca/dataset/sistema-de-informacion-empresarial-mexicano-siem.

## 1. Carga y lmipieza de los Datos

In [1]:
from google.colab import drive
import pandas as pd

# Montar Google Drive en la sesión de Google Colab
drive.mount('/content/drive')

# Definir las rutas a los archivos CSV en Google Drive
ruta_archivo_1 = "/content/drive/MyDrive/Data Nombres Hombre y Mujer/hombres.csv"
ruta_archivo_2 = "/content/drive/MyDrive/Data Nombres Hombre y Mujer/mujeres.csv"
ruta_archivo_3 = "/content/drive/MyDrive/Data Nombres Hombre y Mujer/apellidos.csv"

# Leer los archivos CSV en DataFrames de pandas, utilizando encoding='latin-1' para manejar caracteres especiales
Nombres_Hombres = pd.read_csv(ruta_archivo_1, encoding='latin-1')
Nombres_Mujeres = pd.read_csv(ruta_archivo_2, encoding='latin-1')
Apellidos = pd.read_csv(ruta_archivo_3, encoding='latin-1')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Rellenar los valores NaN en la columna 'apellido' con una cadena vacía
Apellidos['apellido'] = Apellidos['apellido'].fillna('')

# Convertir la primera letra de cada palabra en la columna 'nombre' a mayúscula y el resto a minúscula (título)
Nombres_Hombres['nombre'] = Nombres_Hombres['nombre'].str.title()
Nombres_Mujeres['nombre'] = Nombres_Mujeres['nombre'].str.title()

# Convertir la primera letra de cada palabra en la columna 'apellido' a mayúscula y el resto a minúscula (título)
Apellidos['apellido'] = Apellidos['apellido'].str.title()

# Corregir nombres femeninos que contienen 'Ma!' reemplazando 'Ma!' por 'Ma'
Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('Ma!'), 'nombre'] = Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('Ma!'), 'nombre'].str.replace('Ma!', 'Ma')

# Corregir nombres femeninos que contienen '%O' reemplazando '%O' por 'ño'
Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('%'), 'nombre'] = Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('%'), 'nombre'].str.replace('%O', 'ño')


In [3]:
import re

def contiene_caracteres_raros(cadena):
    patron_espanol = re.compile(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ.\s]')
    return not all(patron_espanol.match(caracter) for caracter in cadena)

# Filtrar las filas que contienen caracteres raros en la columna 'apellido'
filas_con_caracteres_raros = Apellidos[Apellidos['apellido'].apply(contiene_caracteres_raros)]

# Reemplazar todos los caracteres raros por una sola 'ñ' minúscula
Apellidos.loc[filas_con_caracteres_raros.index, 'apellido'] = 'ñ'


In [4]:
# Definir la ruta al archivo CSV que contiene los nombres de personas morales
ruta_archivo_4 = "/content/drive/MyDrive/Data Nombres Hombre y Mujer/datosgob_SIEM2023.csv"

# Leer el archivo CSV y seleccionar la columna 'Razón Social' como serie de pandas
Nombres_Morales = pd.read_csv(ruta_archivo_4, encoding='latin-1')['Razón Social']

# Crear un nuevo DataFrame con la columna 'NombreCompleto' eliminando valores NaN y reiniciando el índice
Nombres_Morales = pd.DataFrame({'NombreCompleto': Nombres_Morales.dropna()}).reset_index(drop=True)

# Convertir la primera letra de cada palabra en la columna 'NombreCompleto' a mayúscula y el resto a minúscula (título)
Nombres_Morales['NombreCompleto'] = Nombres_Morales['NombreCompleto'].str.title()


<ipython-input-4-f84cc6798974>:5: DtypeWarning: Columns (1,2,3,4,6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  Nombres_Morales = pd.read_csv(ruta_archivo_4, encoding='latin-1')['Razón Social']


## 2. Generación del Dataset Empleado para el Entrenamiento

In [5]:
from tqdm import tqdm
import numpy as np
import pandas as pd

# Crear un DataFrame vacío con una columna llamada 'NombreCompleto'
Nombres_Fisicos = pd.DataFrame(columns=['NombreCompleto'])

# Número de registros a generar
num_registros = 100000

# Iniciar el bucle con tqdm para mostrar una barra de progreso con el número total de registros
with tqdm(total=num_registros, desc="Generando registros") as pbar:
    # Iterar sobre el número de registros especificado
    for _ in range(num_registros):
        # Elegir aleatoriamente entre 'Hombre' y 'Mujer' con una probabilidad igual
        genero = np.random.choice(['Hombre', 'Mujer'], p=[0.5, 0.5])

        # Según el género elegido, seleccionar un nombre aleatorio basado en las frecuencias de nombres
        if genero == 'Hombre':
            nombre = np.random.choice(Nombres_Hombres['nombre'], p=(Nombres_Hombres['frec'] / Nombres_Hombres['frec'].sum()).values)
        else:
            nombre = np.random.choice(Nombres_Mujeres['nombre'], p=(Nombres_Mujeres['frec'] / Nombres_Mujeres['frec'].sum()).values)

        # Elegir aleatoriamente entre 'frec_pri' y 'frec_seg' para los apellidos
        apellido_pri = np.random.choice(Apellidos['apellido'], p=(Apellidos['frec_pri'] / Apellidos['frec_pri'].sum()).values)
        apellido_seg = np.random.choice(Apellidos['apellido'], p=(Apellidos['frec_seg'] / Apellidos['frec_seg'].sum()).values)

        # Concatenar nombre y apellidos para formar el nombre completo
        nombre_completo = f"{nombre} {apellido_pri} {apellido_seg}"

        # Crear un DataFrame temporal con el nombre completo generado
        df_temporal = pd.DataFrame({'NombreCompleto': [nombre_completo]})

        # Concatenar el DataFrame temporal al DataFrame principal Nombres_Fisicos
        Nombres_Fisicos = pd.concat([Nombres_Fisicos, df_temporal], ignore_index=True)

        # Actualizar la barra de progreso
        pbar.update(1)

# Mostrar el DataFrame Nombres_Fisicos que contiene los nombres completos generados
Nombres_Fisicos


Generando registros: 100%|██████████| 100000/100000 [06:18<00:00, 264.28it/s]


,NombreCompleto
0,Mónica Vera Castillo
1,Marcelina Pérez Contreras
2,Ana Bertha Mercado Cruz
3,Armando Valdepeña Sánchez
4,Estela Becerra Millán
...,...
99995,Israel Pacheco González
99996,Francelia Gómez Flores
99997,Susana Salazar Madrid
99998,Irene Villanueva Riestra


In [6]:
Nombres_Fisicos['Etiqueta'] = 'P. Física'
Nombres_Morales['Etiqueta'] = 'P. Moral'
Nombres_DataSet = pd.concat([Nombres_Fisicos, Nombres_Morales])

## 3. Creación de Características y Entrenamiento del Modelo

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear un objeto TfidfVectorizer para la extracción de características
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, encoding='latin-1', decode_error='ignore')

# Aplicar el TfidfVectorizer para transformar los nombres completos en características
features = tfidf_vectorizer.fit_transform(Nombres_DataSet['NombreCompleto'])


In [8]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
# X_train: Conjunto de características de entrenamiento
# X_test: Conjunto de características de prueba
# y_train: Etiquetas de entrenamiento
# y_test: Etiquetas de prueba
X_train, X_test, y_train, y_test = train_test_split(features, Nombres_DataSet['Etiqueta'], test_size=0.2, random_state=42)


In [9]:
from sklearn.naive_bayes import MultinomialNB

# Inicialización del modelo Naive Bayes Multinomial
Modelo = MultinomialNB()

# Entrenamiento del modelo Naive Bayes Multinomial
Modelo.fit(X_train, y_train)


MultinomialNB()

## 4. Evaluación del  Modelo

In [10]:
from sklearn import metrics

# Realizar predicciones en el conjunto de prueba
y_pred = Modelo.predict(X_test)

# Calcular y mostrar la precisión del modelo
print("Precisión:", metrics.accuracy_score(y_test, y_pred))



Precisión: 0.9807112015070937


In [11]:
from sklearn.metrics import confusion_matrix

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)


Matriz de Confusión:
[[19889    76]
 [ 1081 38937]]


In [12]:
from sklearn.metrics import classification_report

# Calcular el reporte de clasificación
class_report = classification_report(y_test, y_pred)
print("Reporte de Clasificación:")
print(class_report)


Reporte de Clasificación:
              precision    recall  f1-score   support

   P. Física       0.95      1.00      0.97     19965
    P. Moral       1.00      0.97      0.99     40018

    accuracy                           0.98     59983
   macro avg       0.97      0.98      0.98     59983
weighted avg       0.98      0.98      0.98     59983



## 5. Aplicación del Modelo

In [13]:
nueva_cadena = ['Eduardo Elías Flores Solís']
nueva_cadena_tfidf = tfidf_vectorizer.transform(nueva_cadena)
prediccion = Modelo.predict(nueva_cadena_tfidf)
print('Predicción:', prediccion)

Predicción: ['P. Física']


In [14]:
nueva_cadena = ['Kapitalizer SAPI de SOFOM']
nueva_cadena_tfidf = tfidf_vectorizer.transform(nueva_cadena)
prediccion = Modelo.predict(nueva_cadena_tfidf)
print('Predicción:', prediccion)

Predicción: ['P. Moral']


In [15]:
# Lista de empresas mexicanas que cotizan en la BMV y de compositores mexicanos,nótese que es practicamente imposible que estas cadenas se encuentren en el conjunto entrenamiento,
# por lo que son ideneas para poner a prueba la eficacia del modelo.
nombres = [

    'Grupo Bimbo, SA de CV',
    'Consuelo Velázquez Torres',
    'Grupo Cuauhtémoc Moctezuma, S.A. de C.V.',
    'Empacadora de Frutos y Jugos, S.A.',
    'Banco Mercantil del Norte, S.A.',
    'Manuel María Ponce Cuéllar',
    'José Juventino Rosas Cadenas',
    'Jaime Eduardo Vladimiro Mata Asiain',
        'Teléfonos de México, S.A.B. de C.V.',
    'Carlos Antonio de Padua Chávez y Ramírez'

]

# Crear el DataFrame
df_prueba = pd.DataFrame({'Nombre': nombres})

## Cadenas de Texto Clasificadas por el Modelo
### Prueba de la Eficiencia del Modelo Clasificador

In [16]:
df_prueba['Predicción'] = Modelo.predict(tfidf_vectorizer.transform(df_prueba['Nombre']))
df_prueba

,Nombre,Predicción
0,"Grupo Bimbo, SA de CV",P. Moral
1,Consuelo Velázquez Torres,P. Física
2,"Grupo Cuauhtémoc Moctezuma, S.A. de C.V.",P. Moral
3,"Empacadora de Frutos y Jugos, S.A.",P. Moral
4,"Banco Mercantil del Norte, S.A.",P. Moral
5,Manuel María Ponce Cuéllar,P. Física
6,José Juventino Rosas Cadenas,P. Física
7,Jaime Eduardo Vladimiro Mata Asiain,P. Física
8,"Teléfonos de México, S.A.B. de C.V.",P. Moral
9,Carlos Antonio de Padua Chávez y Ramírez,P. Física
